<img style="float: right;" src="img/python.png">

# Verbindung zu einer MySQL (MariaDB) Datenbank
Als Erstes muss eine Verbindung zu einer Datenbank hergestellt werden. Dazu müssen 2 neue Bibliotheken der Python-Umgebung hinzugefügt werden:
- mysql
- mysql-connector-python

In [1]:
import mysql.connector as mc
import mysql
try:
    connection = mc.connect(
        host='localhost',
        user='root',
        password='12345',
    )
except mc.errors.ProgrammingError as e:
    print("Es ist folgende Fehler aufgetreten: \n ", e)

### Erstellen des Cursors
Hier wird ein Cursor mit den Namen "**c**" erstellt. Danach werden alle Tabellen, auf die der Benutzer Berechtigungen besitzt aufgelistet.

In [2]:
c = connection.cursor()

### Erstellen einer neuen Datenbank auf dem MySQL-Server

In [3]:
try:
    c.execute("CREATE DATABASE telefonbuch")
    print("Datenbank wurde erfolgreich angelegt")
except mysql.connector.Error as e:
    print("Es ist ein Fehler aufgetreten: \n ", e)
finally:
    connection.close()
    

Datenbank wurde erfolgreich angelegt


### Erstellen einer neuen Tabelle
Mithilfe des Befehls "**CREATE TABLE**" wird eine neue Tabelle erstellt:

In [5]:

import mysql.connector as mc
import mysql
from sys import exit

try:
    connection = mc.connect(
        host='localhost',
        user='root',
        password='12345',
        db='telefonbuch'
    )
except mc.errors.ProgrammingError as e:
    print("Es ist folgende Fehler aufgetreten: \n ", e)
    exit(2)
c = connection.cursor()

sql = """CREATE TABLE IF NOT EXISTS telefonbuch (
id INT(6) UNSIGNED AUTO_INCREMENT PRIMARY KEY,
vorname VARCHAR(30) NOT NULL,
nachname VARCHAR(30) NOT NULL,
telefonnummer VARCHAR(30) NOT NULL)
"""
try: 
    c.execute(sql)
    
except:
    print("Es ist folgender Fehler aufgetreten")
    exit(1)
    

('telefonbuch',)


### Vorhandene Tabellen anzeigen

In [ ]:
c.execute("SHOW TABLES")
for table in c:
    print(table)

Anstatt "**CREATE TABLE IF NOT EXISTS telefonbuch**" kann auch "**CREATE TABLE telefonbuch**" verwendet werden. Durch "**IF NOT EXISTS**" wird aber eine Fehlermeldung vermieden, sollte die Tabelle bereits existieren.  

### Werte hinzufügen
Jetzt fügen wir einen Eintrag hinzu:

In [9]:
import mysql.connector as mc
import mysql
from sys import exit

try:
    connection = mc.connect(
        host='localhost',
        user='root',
        password='12345',
        db='telefonbuch'
    )
    c = connection.cursor()
except mc.errors.ProgrammingError as e:
    print("Es ist folgende Fehler aufgetreten: \n ", e)
    exit(2)

vor = input("Bitte geben Sie Vorname ein: ")
nach = input("Bitte geben Sie Nachname ein: ")
telefon = input("Bitte geben Sie Telefonnummer ein: ")

sql = """INSERT INTO telefonbuch (vorname, nachname, telefonnummer) VALUES (%s, %s, %s)"""
params = (vor, nach, telefon)

c.execute(sql, params)
connection.commit()
connection.close()


In Python's **mysql-connector-python** Bibliothek wird **%s** als Platzhalter in SQL-Statements verwendet. Es dient dazu, Werte in das SQL-Statement einzufügen, ohne das Risiko einer SQL-Injection zu erhöhen. Der Platzhalter %s wird unabhängig vom Datentyp des einzufügenden Wertes verwendet. Das bedeutet, **%s wird sowohl für Strings, Zahlen, Datumsangaben als auch andere Datentypen genutzt**.

Es ist wichtig zu beachten, dass der Platzhalter %s in diesem Kontext nichts mit dem String-Formatierungs-Operator in Python zu tun hat. Im Kontext des MySQL-Connectors ist %s einfach ein generischer Platzhalter für Parameter.

Wenn Sie ein SQL-Statement vorbereiten und %s als Platzhalter verwenden, sollten Sie die tatsächlichen Werte als Tupel (oder Liste) an die execute()-Methode übergeben. Der MySQL-Connector sorgt dann dafür, dass diese Werte sicher in das SQL-Statement eingebunden werden, um SQL-Injection zu vermeiden.

Es gibt in diesem Zusammenhang keine anderen Platzhaltertypen wie %d oder %f, die man aus der normalen String-Formatierung in Python kennt. %s wird universell für alle Datentypen verwendet.

### Datenbank auslesen
Jetzt schauen wir nach, welche Einträge in der Tabelle vorhanden sind:


In [11]:
import mysql.connector as mc
import mysql
from sys import exit

try:
    connection = mc.connect(
        host='localhost',
        user='root',
        password='12345',
        db='telefonbuch'
    )
    c = connection.cursor()
except mc.errors.ProgrammingError as e:
    print("Es ist folgende Fehler aufgetreten: \n ", e)
    exit(2)
    
sql = """SELECT * FROM telefonbuch"""
c.execute(sql)
ausgabe = c.fetchall()
connection.close()
print("Ausgabe als Liste")
print(ausgabe)

print("Iteration")
for i in ausgabe:
    print(i)

print("Iteration noch mal")    
for _, name, nachname, telefon in ausgabe:
    print(name, nachname, telefon)

Ausgabe als Liste
[(1, 'Susanne', 'Smith', '049785'), (2, 'Tom', 'Norton', '2354'), (3, 'Ceciele', 'Swan', '01245')]
Iteration
(1, 'Susanne', 'Smith', '049785')
(2, 'Tom', 'Norton', '2354')
(3, 'Ceciele', 'Swan', '01245')
Iteration noch mal
Susanne Smith 049785
Tom Norton 2354
Ceciele Swan 01245


### Daten aktualisieren

Die Rufnummer eines Eintrags im Telefonbuch hat sich geändert. Diese müssen wir anpassen:

In [12]:

import mysql.connector as mc
import mysql
from sys import exit

try:
    connection = mc.connect(
        host='localhost',
        user='root',
        password='12345',
        db='telefonbuch'
    )
    c = connection.cursor()
except mc.errors.ProgrammingError as e:
    print("Es ist folgende Fehler aufgetreten: \n ", e)
    exit(2)

vorname = "Tom"
nachname = "Norton"
neue_rufnummer = "1236547899"

params = (neue_rufnummer, vorname, nachname, )
sql = """UPDATE telefonbuch SET telefonnummer = %s WHERE vorname = %s AND nachname = %s """

c.execute(sql, params)
connection.commit()
connection.close()




### Wir löschen einen Eintrag. 

Als Erstes stellen wir sicher, dass wir den richtigen Benutzer löschen. Dafür brauchen wir am besten eine Angabe, die eindeutig ist.

Da wir vermeiden wollen, Code mehrfach zu schreiben suchen wir uns einen Eintrag aus, indem wir die Datenbank-Abfrage-Zelle ggf. erneut ausführen, und suchen uns die **id** des Eintrags, den wir löschen wollen.


In [1]:
import mysql.connector as mc
import mysql
from sys import exit

try:
    connection = mc.connect(
        host='localhost',
        user='root',
        password='12345',
        db='telefonbuch'
    )
    c = connection.cursor()
except mc.errors.ProgrammingError as e:
    print("Es ist folgende Fehler aufgetreten: \n ", e)
    exit(2)
    
sql = """SELECT * FROM telefonbuch"""
c.execute(sql)
ausgabe = c.fetchall()

print("Welchen Eintrag möchten Sie löschen?")    
for id, name, nachname, telefon in ausgabe:
    print(id, name, nachname, telefon)
    
auswahl = input("Bitte geben Sie die Kennnummer ein: ")

sql = """DELETE FROM telefonbuch WHERE id=%s"""
params = (auswahl, )

frage = input(f"Sind Sie sicher, dass Sie den Eintrag mit der ID {auswahl} löschen möchten (j/n)?").lower()

if frage == "j":
    c.execute(sql, params)
    connection.commit()
    connection.close()
else:
    connection.close()
    exit(0) # Führt in Jupyter Notebooks zu Fehlern


Welchen Eintrag möchten Sie löschen?
1 Susanne Smith 049785
4 Tom Norton 01245
5 Bella Neue 04578
6 Ronald Horton 01245


In der Zelle "**Datenbank auslesen**" können wir uns vergewissern, ob der Vorgang erfolgreich war.

### Eine keine Aufgabe:
Bitte erstellen Sie einen kleinen Warenkatalog. in diesen Katalog legen Sie bitte Artikel mit den folgenden Merkmalen an:

- ID (primary_key)
- Produkt Name
- Artikelnummer
- Netto-Preis

Schreiben Sie bitte erstmal nur die Erstellung der Datenbank "**warenkatalog**", das Anlegen der benötigten Tabelle und das Hinzufügen der Artikel. Weitere Funktionen können Sie heute Nachmittag in der Übungszeit programmieren.
 


In [4]:
import mysql.connector as mc
import mysql
from sys import exit


def add_data(cursor, connection, name, nummer, price):
    sql = """INSERT INTO katalog (Produktname, Artikelnummer, Netto_Price) VALUES (%s, %s, %s)"""
    params = (name, nummer, price)
    cursor.execute(sql, params)
    
def show_data(cursor):
    sql = """SELECT * FROM katalog"""
    cursor.execute(sql)
    result = cursor.fetchall()
    print("*" * 40)
    for id, name, nummer, price in result:
        print(f"{id} \t {name} \t{nummer} \t{price}")
    print("*" * 40)
    
def menu():
    print(
        "\tA / a: Vorhandene Tabellen anzeigen\n",
        "\tB / b: Daten hinzufügen\n"
        "\tE / e: End"
    )

def main():
    try:
        connection = mc.connect(
        host='localhost',
        user='root',
        password='12345',
        )
        
    except mc.errors.ProgrammingError as e:
        print("Es ist folgende Fehler aufgetreten: \n ", e)
    
    try:
        cursor = connection.cursor()
        cursor.execute("CREATE DATABASE IF NOT EXISTS store")
        cursor.execute("USE store")
        sql = """CREATE TABLE IF NOT EXISTS katalog(
        id INT AUTO_INCREMENT PRIMARY KEY,
        Produktname VARCHAR(30) NOT NULL,
        Artikelnummer VARCHAR(30) NOT NULL,
        Netto_Price FLOAT (7, 2))"""
        cursor.execute(sql)
    except mc.errors.ProgrammingError as e:
        print("Es ist folgende Fehler aufgetreten: \n ", e)
    
    while True:
        menu()
        ui = input("-> ").upper()
        if ui == "E":
            break
        elif ui == "A":
            show_data(cursor)
        elif ui == "B":
            name = input("Produktname: ")
            nummer = input("Artikelnummer: ")
            price = input("Netto_Price: ")
            add_data(cursor, connection, name, nummer, price)
            
    connection.close()
    
if __name__ == "__main__":
    main()

    
    

	A / a: Vorhandene Tabellen anzeigen
 	B / b: Daten hinzufügen
	E / e: End
	A / a: Vorhandene Tabellen anzeigen
 	B / b: Daten hinzufügen
	E / e: End
****************************************
****************************************
	A / a: Vorhandene Tabellen anzeigen
 	B / b: Daten hinzufügen
	E / e: End
	A / a: Vorhandene Tabellen anzeigen
 	B / b: Daten hinzufügen
	E / e: End
	A / a: Vorhandene Tabellen anzeigen
 	B / b: Daten hinzufügen
	E / e: End
	A / a: Vorhandene Tabellen anzeigen
 	B / b: Daten hinzufügen
	E / e: End


## Eine weitere Spalte der Tabelle hinzufügen

Pläne ändern sich. Vielleicht müssen Sie ihre bereits erstellte Tabelle um eine Spalte erweitern.

Um eine vorhandene Tabelle in einer MySQL-Datenbank um eine weitere Spalte zu erweitern, verwenden Sie das SQL-Statement ALTER TABLE. In Python können Sie dieses Statement mit einem MySQL-Connector ausführen.

Dafür gehen Sie so vor:

In [5]:
import mysql.connector as mc
import mysql
from sys import exit
try:
    connection = mc.connect(
        host='localhost',
        user='root',
        password='12345',
        db='warenkatalog'
    )
    c = connection.cursor()
except mc.errors.ProgrammingError as e:
    print("Es ist folgende Fehler aufgetreten: \n ", e)
    exit(2)
    
tabelle = "products"
new_column_name = "stock_new"
column_type = "VARCHAR(30)"

try:
    c.execute(f"ALTER TABLE {tabelle} ADD COLUMN {new_column_name} {column_type}")
except mysql.connector.Error as err:
    print("Es ist ein Fehler aufgetreten:", err)

finally:
    connection.close()

## Wildcards

Nicht immer wissen wir genau, wie ein Datensatz genau heißt. In solchen Fällen sind **Wildcards** das Mittel der Wahl.

Die Verwendung von Wildcards in SQL ist eine effektive Methode, um nach Daten zu suchen, wenn der genaue Name oder Wert nicht bekannt ist. In SQL wird häufig der Wildcard-Charakter **%** in Kombination mit dem **LIKE**-Operator verwendet, um teilweise Übereinstimmungen in einer Zeichenfolge zu finden. 

Damit wir ein wenig ausprobieren können, müssen wir noch ein paar Produkte in unserer Warendatenbank anlegen. Vielleicht eine gewisse Anzahl verschiedene Stühle und Tische. :-)

<img style="float: center;" src="img/datensatz.png"><br>

#### Suche nach einem bestimmten Muster am Anfang:


Um den Code ein wenig zu kürzen, erstellen wir eine Verbindungs-Funktion:

In [7]:
def create_connection():
    connection = mc.connect(
        host='localhost',
        user='root',
        password='12345',
        db='warenkatalog'
    )
    c = connection.cursor()
    
    return connection, c

Wenn Sie nach Einträgen suchen möchten, die mit einem bestimmten Muster beginnen, setzen Sie die %-Wildcard **nach** dem Muster.

In [11]:
def search_pattern(pattern):
    connection, c = create_connection()
    query = """SELECT * FROM products WHERE title LIKE %s"""
    c.execute(query, (pattern + '%',))
    result = c.fetchall()
    
    for row in result:
        print(row)
    
search_pattern("chair")

(4, 'chair', 'Big company', '150', '21', '123-658', None)
(6, "chair 'Star'", 'Big company', '187', '54', '25-65', None)


Wenn Sie nach Einträgen suchen möchten, die mit einem bestimmten Muster beginnen, setzen Sie die %-Wildcard **vor** dem Muster.

In [15]:
def search_pattern_ends_with(pattern):
    connection, c = create_connection()
    query = """SELECT * FROM products WHERE title LIKE %s"""
    c.execute(query, ('%' + pattern,))
    result = c.fetchall()
    
    for row in result:
        print(row)
    
search_pattern_ends_with("chair")

(4, 'chair', 'Big company', '150', '21', '123-658', None)
(9, 'kitchenchair', 'Kitchen', '25', '36', '325-231', None)


Wenn Sie nach Einträgen suchen möchten, die ein bestimmtes Muster irgendwo enthalten, setzen Sie die %-Wildcard **vor und nach** dem Muster.

In [18]:
def search_pattern_vor_und_nach(pattern):
    connection, c = create_connection()
    query = """SELECT * FROM products WHERE title LIKE %s"""
    c.execute(query, ('%' + pattern + '%',))
    result = c.fetchall()
    
    for row in result:
        print(row)
    
search_pattern_vor_und_nach("chair")

(4, 'chair', 'Big company', '150', '21', '123-658', None)
(6, "chair 'Star'", 'Big company', '187', '54', '25-65', None)
(9, 'kitchenchair', 'Kitchen', '25', '36', '325-231', None)


Suche nach Produkten mit einem **spezifischen Muster** an einer bestimmten Stelle:

In [21]:
def search_special_pattern(pattern):
    connection, c = create_connection()
    query = """SELECT * FROM products WHERE title LIKE %s"""
    c.execute(query, (pattern,))
    result = c.fetchall()
    
    for row in result:
        print(row)
    
search_special_pattern("ch_ir")

(4, 'chair', 'Big company', '150', '21', '123-658', None)
(10, 'chuir', 'Tree', '17', '12', '236-789', None)


## Datapipelines

#### Eine Sache noch zum Schluss

Data Pipelines im Kontext von SQL und Datenbanken beziehen sich im Allgemeinen auf **automatisierte Prozesse**, die für die systematische Übertragung und Transformation von Daten von einer Quelle zu einem Ziel verwendet werden. Diese Pipelines spielen eine entscheidende Rolle in der Datenverarbeitung und Business Intelligence. Sie ermöglichen es, große Mengen von Daten effizient und zuverlässig zu bewegen, zu verarbeiten und für Analysezwecke nutzbar zu machen.

Hier sind einige Kernaspekte von Data Pipelines:

- **Datenextraktion (Extraction)**: Dies ist der erste Schritt, bei dem Daten aus einer oder mehreren Quellen extrahiert werden. Diese Quellen können Datenbanken, Dateien, APIs, externe Dienste und andere sein.

- **Datenverarbeitung/-transformation (Transformation)**: Nach der Extraktion werden die Daten häufig transformiert. Diese Transformationen können das Bereinigen von Daten, das Anwenden von Geschäftslogik, das Aggregieren, das Sortieren, das Joinen mit anderen Datensätzen usw. umfassen. Ziel ist es, die Daten in ein Format zu bringen, das für die Analyse oder den nächsten Schritt in der Pipeline geeignet ist.

- **Datenspeicherung (Loading)**: Im letzten Schritt werden die verarbeiteten Daten in einem Zielsystem gespeichert, das ein Data Warehouse, eine Datenbank oder ein anderes Speichersystem sein kann. Dieser Prozess wird oft als ETL-Prozess (Extraction, Transformation, Loading) bezeichnet.

- **Automatisierung**: Data Pipelines sind oft automatisiert, um kontinuierlich oder zu geplanten Zeiten zu laufen. Dies ist besonders wichtig, um die Aktualität der Daten in schnelllebigen Geschäftsumgebungen zu gewährleisten.

- **Überwachung und Wartung**: Da Data Pipelines kritische Geschäftsprozesse unterstützen können, ist ihre ständige Überwachung und regelmäßige Wartung wichtig, um Probleme wie Datenverlust, Verzögerungen oder Fehler in der Datenverarbeitung zu verhindern.

- **SQL in Data Pipelines**: SQL spielt eine entscheidende Rolle in vielen Phasen einer Data Pipeline, insbesondere bei der Extraktion von Daten aus relationalen Datenbanken und bei deren Transformation. SQL wird wegen seiner Effizienz und Effektivität bei der Datenmanipulation und -abfrage in solchen Szenarien häufig eingesetzt.

Data Pipelines sind ein wesentlicher Bestandteil von Data-Warehousing-Lösungen, Business-Intelligence-Systemen, Data-Lake-Implementierungen und anderen datenintensiven Anwendungen, die auf der Analyse und dem Reporting von Daten basieren.

<img style="float: center;" src="img/wbs-logo.jpg"><br>
Author: Dirk Maric

### Abbildungs- und Quellenverzeichnis

Das Python Logo ist ein eingetragenes Warenzeichen der Python Software Foundation
Alle auf dieser Website veröffentlichten Logos sowie Marken-, Produkt- und Warenzeichen sind Eigentum der jeweiligen Unternehmen
© WBS TRAINING AG – Alle Rechte vorbehalten

### Nutzungsrechte:
Die Nutzung dieser Dokumentation ist ausschließlich für Schulungszwecke der WBS TRAINING AG gestattet. Eine Weitergabe an Dritte, auch auszugsweise, sowie Vervielfältigungen und Verbreitungen aller Art (elektronische und andere Verfahren) inklusive Übersetzungen sind nur mit vorheriger schriftlicher Zustimmung des Rechtinhabers gestattet. Zuwiderhandlungen verpflichten zu Schadenersatz.

### Herausgeber:

WBS TRAINING AG
Lorenzweg 5
12099 Berlin
Haftungsausschluss:
Alle Inhalte sind nach bestem Wissen korrekt und vollständig recherchiert und mit größtmöglicher Sorgfalt für die Schulungsunterlage zusammengestellt. Wir sind um die laufende Aktualisierung aller Informationen und Daten bemüht. Dennoch können Fehler (z.B. Abweichungen zur beschriebenen Hard- und Software durch kurzfristige Updates) auftreten, sodass wir für die vollständige Übereinstimmung, Richtigkeit und Aktualität keine Gewähr übernehmen. Hinweise unserer Nutzer werden konsequent weiterverfolgt.
